In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import BooleanVar, Checkbutton, messagebox
from PIL import Image, ImageTk
import numpy as np

### Sciąganie potrzebnych danych:

In [8]:
filename = 'merged_weather_prices_data.csv'

panel_data = {
    'SunPower X21-345': {'cost': 80, 'output_watt': 500, 'tariff': 0.18, 'temperature_coefficient': -0.3},
    'LG NeON R': {'cost': 70, 'output_watt': 510, 'tariff': 0.16, 'temperature_coefficient': -0.29},
    'Canadian Solar HiKu': {'cost': 85, 'output_watt': 550, 'tariff': 0.14, 'temperature_coefficient': -0.35},
    'REC Alpha Series': {'cost': 80, 'output_watt': 510, 'tariff': 0.17, 'temperature_coefficient': -0.32},
    'Trina Solar Tallmax': {'cost': 75, 'output_watt': 500, 'tariff': 0.15, 'temperature_coefficient': -0.31}
}

def load_hourly_cloudiness(filename):
    df = pd.read_csv(filename, delimiter=',')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
    df['Date'] = df['Timestamp'].dt.date
    hourly_data = df.groupby(['City', 'Date']).agg({'Cloudiness': 'mean', 'Temperature': 'mean', 'Elevation': 'mean', 'Azimuth': 'mean', 'prcp': 'mean', 'snow': 'mean'}).reset_index()
    return hourly_data

cloudiness_data = load_hourly_cloudiness(filename)

### Parametr **Precipitation**, parametr **Cloudiness**, parametr **Tilt Angle**

#### Aktualizacja Stanu Paneli w Zależności od Opadów atmosferycznych:

- Jeśli panel jest brudny, wydajność zmniejsza się o 30%.
- Jeśli pada deszcz, wydajność zmniejsza się o dodatkowe 30% przez jeden dzień.
- Jeśli pada śnieg, wydajność zmniejsza się o dodatkowe 60% przez odpowiednio 2 lub 3 dni.


In [9]:
days_without_precipitation = 0
snow_effect_days = 0
rain_effect_days = 0
dirty_panel = False

def calculate_energy(tilt_angle, panel_azimuth, elevations, azimuths, cloudiness, rain, snow):
    energy_angel = 0  # całkowita energia w kWh
    
    global days_without_precipitation
    global snow_effect_days
    global rain_effect_days
    global dirty_panel

    for solar_elevation, solar_azimuth in zip(elevations, azimuths):

        # Aktualizacja stanu paneli w zależności od pogody
        if rain > 0:
            days_without_precipitation = 0
            rain_effect_days += 1
            dirty_panel = False  # Panel jest umyty po deszczu
            
        elif snow > 0:
            if snow < 30:
                if snow_effect_days == 0:
                    snow_effect_days += 2
                else:
                    snow_effect_days += 1
                
            else:
                if snow_effect_days == 0:
                    snow_effect_days += 3
                else:
                    snow_effect_days += 2
                
            days_without_precipitation = 0
            dirty_panel = False  # Panel jest umyty po stopieniu śniegu
            
        else:
            dirty_panel = True
            days_without_precipitation += 1
            if days_without_precipitation >= 30:
                dirty_panel = False
                days_without_precipitation = 0  # Reset po umyciu panelu
                

        if solar_elevation is not None and solar_azimuth is not None:
            # Sprawdzamy, czy panel na wschód czy na zachód powinien produkować energię
            if (panel_azimuth == 90 and solar_azimuth > 180) or (panel_azimuth == 270 and solar_azimuth < 180):
                continue  # Jeśli panel wschodni i azymut > 180 lub panel zachodni i azymut < 180, to nie produkuje energii

            tilt_angle_rad = np.radians(tilt_angle)
            solar_elevation_rad = np.radians(solar_elevation)
            solar_azimuth_rad = np.radians(solar_azimuth)

            azimuth_difference_rad = np.abs(solar_azimuth_rad - np.radians(panel_azimuth))

            cos_incidence = (np.sin(solar_elevation_rad) * np.cos(tilt_angle_rad) +
                             np.cos(solar_elevation_rad) * np.sin(tilt_angle_rad) * np.cos(azimuth_difference_rad))

            # Mnożenie przez ilość godzin słonecznych i moc panelu (1 kWp)
            energy = cos_incidence / len(elevations)  # dzielenie przez liczbę pomiarów, aby dostać średnią dzienną wartość

            # Uwzględnienie zachmurzenia
            efficiency_factor = (1 - cloudiness / 100) * (1/3) + (2/3)  # Przykład: przy zachmurzeniu 100%, efektywność to 1/3
            energy *= efficiency_factor

            # Uwzględnienie brudu
            if dirty_panel:
                energy *= 0.7  # brudne panele mają 30% mniejszą wydajność

            if rain_effect_days > 0:
                energy *= 0.7  # deszcz zmniejsza wydajność o 30%
                rain_effect_days -= 1
            # Uwzględnienie efektu śniegu
            if snow_effect_days > 0:
                energy *= 0.4  # śnieg zmniejsza wydajność o 60%
                snow_effect_days -= 1

            energy_angel += max(0, energy)  # Zapewnienie, że energia nie jest ujemna
    return energy_angel

### **Wykres**, parametr **Orientation**, parametr **Temperature Effect**, 
### parametr **Panel Type**, parametr **Location**:

In [10]:
def calculate_temperature_coefficient(panel_type):
    return panel_data[panel_type]['temperature_coefficient']

def generate_chart(right_panel, location, daily_data, budget, panel_type, tilt_angle_var, maintenance_var, orientation, use_fixed_price, fixed_price, maintenance_cost=10):
    global chart_canvas, fig

    cost_per_panel = panel_data[panel_type]['cost']
    panel_output_watt = panel_data[panel_type]['output_watt']
    temperature_coefficient = calculate_temperature_coefficient(panel_type)
    output_at_25_degrees = panel_output_watt

    if use_fixed_price:
        try:
            tariff_per_kwh = float(fixed_price)
        except ValueError:
            tk.messagebox.showerror("Invalid Input", "Please enter a valid number for the fixed price.")
            return
    else:
        tariff_per_kwh = panel_data[panel_type]['tariff']

    number_of_panels = budget // cost_per_panel
    total_power_kw = number_of_panels * (panel_output_watt / 1000)
    tilt_angle = tilt_angle_var.get()  # Pobierz wartość kąta nachylenia z suwaka

    location_data = daily_data[daily_data['City'] == location].copy()

    if 'Elevation' not in location_data.columns or 'Azimuth' not in location_data.columns:
        messagebox.showerror("Invalid Data", "The data is missing 'Elevation' and/or 'Azimuth' columns.")
        return

    # Dodajemy sprawdzenie na obecność orientacji
    if orientation not in ['South', 'East-West']:
        messagebox.showerror("Invalid Input", "Please select a valid orientation ('South' or 'East-West').")
        return

    # Tworzymy kolumnę Temp_Effect
    location_data['Temp_Effect'] = 1 - ((location_data['Temperature'] - 25).clip(lower=0) * 0.004)
    location_data['New_Panel_Output'] = output_at_25_degrees + abs(25 - location_data['Temperature']) * temperature_coefficient
    expected_output_kw = total_power_kw * location_data['Temp_Effect'] * ((1 - location_data['Cloudiness'] / 100) + 0.01 * (location_data['Cloudiness'] / 100))
    expected_output_kw *= location_data['New_Panel_Output'] / output_at_25_degrees  # Adjusted for temperature

    if orientation == 'South':
        location_data['Energy'] = location_data.apply(lambda row: calculate_energy(tilt_angle, 180,
                                                                                   np.array([row['Elevation']]), np.array([row['Azimuth']]), row['Cloudiness'], row['prcp'], row['snow']), axis=1)
        expected_output_kw = total_power_kw * location_data['Temp_Effect'] * location_data['Energy']
    elif orientation == 'East-West':
        power_kw_per_side = total_power_kw / 2
        location_data['Energy_East'] = location_data.apply(lambda row: calculate_energy(tilt_angle, 90,
                                                                                        np.array([row['Elevation']]), np.array([row['Azimuth']]), row['Cloudiness'], row['prcp'], row['snow']), axis=1)
        location_data['Energy_West'] = location_data.apply(lambda row: calculate_energy(tilt_angle, 270,
                                                                                        np.array([row['Elevation']]), np.array([row['Azimuth']]), row['Cloudiness'], row['prcp'], row['snow']), axis=1)
        location_data['Energy'] = location_data['Energy_East'] + location_data['Energy_West']
        expected_output_kw = power_kw_per_side * location_data['Temp_Effect'] * location_data['Energy_East'] + \
                             power_kw_per_side * location_data['Temp_Effect'] * location_data['Energy_West']

    location_data['Estimated_Earnings'] = (expected_output_kw - abs(25 - location_data['Temperature']) * temperature_coefficient / 1000) * tariff_per_kwh  # Odejmowanie współczynnika temperatury od uzyskanej energii godzinowej
    location_data['Cumulative_Earnings'] = location_data['Estimated_Earnings'].cumsum() - budget

    if maintenance_var.get():
        location_data['Date'] = pd.to_datetime(location_data['Date'])
        location_data.sort_values('Date', inplace=True)
        location_data['Month'] = location_data['Date'].dt.month
        location_data['Year'] = location_data['Date'].dt.year
        location_data['Month_Change'] = (location_data['Month'].diff() != 0) | (location_data['Year'].diff() != 0)
        location_data.loc[location_data['Month_Change'], 'Maintenance_Deduction'] = maintenance_cost
        location_data['Maintenance_Deduction'].fillna(0, inplace=True)
        location_data['Cumulative_Earnings'] -= location_data['Maintenance_Deduction'].cumsum()

    dates = location_data['Date'].to_numpy()
    cumulative_earnings = location_data['Cumulative_Earnings'].to_numpy()
    
    if chart_canvas is not None:
        chart_canvas.get_tk_widget().destroy()
    if fig is not None:
        plt.close(fig)

    fig = Figure(figsize=(5, 4))
    ax = fig.add_subplot(111)

    # Ustawienie stałych wartości na osiach x i y
    ax.plot(dates, cumulative_earnings, linestyle='-', color='blue', label='Cumulative Earnings')
    ax.set_xlabel('Date')
    ax.set_ylabel('Earnings ($)')

    ax.set_title(f'Cumulative Earnings Over Time - {location} ({orientation})')
    ax.legend()
    ax.grid(True)

    # Ustawienie zakresu osi x i y
    ax.set_xlim(location_data['Date'].min(), location_data['Date'].max())
    ax.set_ylim(-budget, budget)  # Zakres osi y od -(initial investment) do initial investment

    chart_canvas = FigureCanvasTkAgg(fig, master=right_panel)
    chart_canvas.draw()
    chart_canvas.get_tk_widget().pack(fill='both', expand=True)




### Tworzenie Aplikacji:

In [11]:
def setup_main_window(root, bg_image_path):
    
    bg_image = Image.open(bg_image_path)
    bg_photo = ImageTk.PhotoImage(bg_image)
    canvas = tk.Canvas(root, width=1024, height=1024)
    canvas.pack(fill='both', expand=True)
    canvas.create_image(0, 0, image=bg_photo, anchor='nw')
    root.canvas = bg_photo
    return canvas

def update_labels(tilt_angle_var, selected_tilt_label, recommended_tilt_label):
    selected_tilt = tilt_angle_var.get()
    recommended_tilt_range = "20° to 30°"
    selected_tilt_label.config(text=f"Selected Tilt Angle: {selected_tilt}°")
    recommended_tilt_label.config(text=f"Recommended Tilt Range: {recommended_tilt_range}")

def on_button_click(text, update_ui):
    if text == "Exit":
        root.quit()
    elif text == "New Calculations":
        update_ui()
        
def create_buttons(canvas, update_ui):
    button_texts = ["New Calculations", "Settings", "Exit"]
    buttons = []
    for idx, text in enumerate(button_texts):
        btn = tk.Button(canvas, text=text, fg = 'white',bg = 'midnightblue', width=20, height=1, font=("Arial", 14),command=lambda t=text: on_button_click(t, update_ui))
        buttons.append(btn)
        canvas.create_window(512, 300 + idx * 50, anchor='n', window=btn)


def on_generate_button_click(right_panel, maintenance_var, investment_entry, location_var, tilt_angle_var, panel_type_var, orientation_var, fixed_price_var, fixed_price_entry):
    location_str = location_var.get()
    tilt_angle = tilt_angle_var.get()
    panel_type = panel_type_var.get()
    orientation = orientation_var.get()
    use_fixed_price = fixed_price_var.get()
    fixed_price = fixed_price_entry.get().strip()
    generate_chart_with_input_validation(right_panel, maintenance_var, investment_entry, location_str, tilt_angle, panel_type, tilt_angle_var, orientation, use_fixed_price, fixed_price)

global chart_canvas, fig
chart_canvas = None
fig = None

def show_new_calculations(root, maintenance_var):
    new_width, new_height = 1024, 768
    root.geometry(f"{new_width}x{new_height}")

    for widget in root.winfo_children():
        widget.destroy()

    left_panel = tk.Frame(root, width=600, bg='midnightblue')
    left_panel.pack(side='left', fill='y', padx=20, pady=20)

    right_panel = tk.Frame(root, bg='white')
    right_panel.pack(side='left', fill='both', expand=True)

    maintenance_check = Checkbutton(left_panel, text='Enable Monthly Maintenance', var=maintenance_var)
    maintenance_check.config(fg='midnightblue')
    maintenance_check.pack(pady=(10, 5))

    panel_type_var = tk.StringVar(value="SunPower X21-345")
    panel_types = ['SunPower X21-345', 'LG NeON R', 'Canadian Solar HiKu', 'REC Alpha Series', 'Trina Solar Tallmax']
    panel_menu = tk.OptionMenu(left_panel, panel_type_var,  *panel_types)
    panel_menu.config(fg='midnightblue')
    tk.Label(left_panel, text='Select Panel Type:', bg='midnightblue', fg='white').pack(pady=(10, 5))
    panel_menu.pack(pady=(0, 10))

    tk.Label(left_panel, text='Initial Investment ($):', bg='midnightblue', fg='white').pack(pady=(10, 5))
    investment_entry = tk.Entry(left_panel)
    investment_entry.config(fg='midnightblue')
    investment_entry.pack(pady=(0, 10))

    tk.Label(left_panel, text='Select Location:', bg='midnightblue', fg='white').pack(pady=(10, 5))
    location_var = tk.StringVar(left_panel)
    locations = ['Krakow', 'Wroclaw', 'Warsaw']
    location_menu = tk.OptionMenu(left_panel, location_var, *locations)
    location_menu.config(fg='midnightblue')
    location_menu.pack(pady=(0, 10))

    tk.Label(left_panel, text='Select Tilt Angle:', bg='midnightblue', fg='white').pack(pady=(10, 5))
    tilt_angle_var = tk.DoubleVar(value=30)
    tilt_angle_slider = tk.Scale(left_panel, from_=0, to=90, variable=tilt_angle_var, orient=tk.HORIZONTAL, command=lambda x: update_labels(tilt_angle_var, selected_tilt_label))
    tilt_angle_slider.config(fg='midnightblue')
    tilt_angle_slider.pack(pady=(0, 10))

    selected_tilt_label = tk.Label(right_panel, text=f"Selected Tilt Angle: {tilt_angle_var.get()}°")
    selected_tilt_label.pack()
    
    tk.Label(left_panel, text='Select Orientation:', bg='midnightblue', fg='white').pack(pady=(10, 5))
    orientation_var = tk.StringVar(left_panel)
    orientations = ['South', 'East-West']
    orientation_menu = tk.OptionMenu(left_panel, orientation_var, *orientations)
    orientation_menu.config(fg='midnightblue')
    orientation_menu.pack(pady=(0, 10))

    fixed_price_var = BooleanVar()
    fixed_price_check = Checkbutton(left_panel, text='Use Fixed Price', variable=fixed_price_var)
    fixed_price_check.config(fg='midnightblue')
    fixed_price_check.pack(pady=(10, 5))
    

    tk.Label(left_panel, text='Fixed Price (USD/kWh):', bg='midnightblue', fg='white').pack(pady=(10, 5))
    fixed_price_entry = tk.Entry(left_panel)
    fixed_price_entry.config(fg='midnightblue')
    fixed_price_entry.pack(pady=(0, 10))

    generate_button = tk.Button(left_panel, text="Generate Chart", command=lambda: on_generate_button_click(right_panel, maintenance_var, investment_entry, location_var, tilt_angle_var, panel_type_var, orientation_var, fixed_price_var, fixed_price_entry), bg='white', fg='black', width=20)
    generate_button.config(fg = 'midnightblue')
    generate_button.pack(pady=(20, 10))


def generate_chart_with_input_validation(right_panel, maintenance_var, investment_entry_widget, location_str, tilt_angle, panel_type, tilt_angle_var, orientation, use_fixed_price, fixed_price):
    # Ensure a location is selected
    location = location_str
    if not location:
        tk.messagebox.showerror("Invalid Input", "Please select a location.")
        return

    # Retrieve the string value from the Entry widget and strip any whitespace
    investment_value = investment_entry_widget.get().strip()

    # Remove any spaces, commas, or dollar signs that might interfere with numeric conversion
    sanitized_value = investment_value.replace(",", "").replace(" ", "").replace("$", "")

    try:
        # Convert sanitized input to float then int for budget calculation
        initial_investment = int(float(sanitized_value))

        # Generate the chart using the corrected initial investment as the budget
        generate_chart(right_panel, location, cloudiness_data, initial_investment, panel_type, tilt_angle_var, maintenance_var, orientation, use_fixed_price, fixed_price)
    except ValueError as e:
        # If there's an error in converting to integer, display an error message
        tk.messagebox.showerror("Invalid Input", "Please enter a valid number for the initial investment.")
        print(f"Error converting investment value: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Solar Energy ROI Analysis App")
    maintenance_var = BooleanVar()
    canvas = setup_main_window(root, "background.png")
    create_buttons(canvas, lambda: show_new_calculations(root, maintenance_var))
    root.mainloop()


> #### **_Co można poprawić_**
* **Prawdziwy kurs a nie konstanta:**

  - Użycie rzeczywistych cen energii elektrycznej zamiast stałej wartości. Można zintegrować API, które pobiera rzeczywiste lub historyczne godzinowe ceny energii elektrycznej, np. z platformy ENTSO-E Transparency lub Polskiej Giełdy Energii (TGE).
* **Znaleźliśmy anomalie w danych i zmieniliśmy na wartość sąsiada:**

  - Implementacja systemu wykrywania anomalii w danych wejściowych. W przypadku wykrycia anomalii, błędna wartość jest zastępowana wartością sąsiada lub średnią z pobliskich wartości.
* **Kąt nie do końca jest zrobiony:**

  - Udoskonalenie obliczeń optymalnego kąta nachylenia paneli słonecznych na podstawie lokalizacji geograficznej i pory roku, co może znacząco wpłynąć na efektywność produkcji energii.
* **Jakieś warunki dodatkowe: na przykład śnieg itd:**

  - Uwzględnienie dodatkowych warunków środowiskowych, takich jak pokrywa śnieżna, akumulacja brudu i inne czynniki, które mogą wpływać na efektywność paneli słonecznych. Można dodać opcje wejścia użytkownika lub zintegrować dane w czasie rzeczywistym dotyczące tych warunków.